# Mink usage examples

This notebook demonstrates some examples of using mink and how it interacts with sklearn.

## Imports

In [1]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import pandas as pd

In [2]:
from mink import NeuralNetClassifier
from mink import NeuralNetRegressor
from mink.layers import DenseLayer
from mink.layers import InputLayer
from mink.updates import SGD

## MNIST classification task

### Data

In [3]:
X, y = make_classification(n_samples=5000)

### Define neural net

Note: No need to specify the shape of the training data, number of classes, or to set softmax nonlinearity. The `NeuralNetClassifier` class takes care of all of that, as is expected from an sklearn estimator.

In [4]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [5]:
net = NeuralNetClassifier(l2, verbose=1)

If we want to change certain parameters after initialization, just use the `set_params` method and the double-underscore notation known from sklearn.

In [6]:
net.set_params(update__learning_rate=0.5)

NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7facf9ab5fd0>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

### Train

In [7]:
net.fit(X, y, num_epochs=0)

NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7facf9ab5fd0>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [8]:
y_proba = net.predict_proba(X)

In [9]:
(np.argmax(y_proba, 1) == y).mean()

0.54379999999999995

In [10]:
pdb on

Automatic pdb calling has been turned ON


In [11]:
net.fit(X, y, num_epochs=20)

epochs:    1 | loss: 0.34726
epochs:    2 | loss: 0.33814
epochs:    3 | loss: 0.33034
epochs:    4 | loss: 0.32314
epochs:    5 | loss: 0.31696
epochs:    6 | loss: 0.31145
epochs:    7 | loss: 0.30663
epochs:    8 | loss: 0.30156
epochs:    9 | loss: 0.29680
epochs:   10 | loss: 0.29266
epochs:   11 | loss: 0.28831
epochs:   12 | loss: 0.28442
epochs:   13 | loss: 0.28018
epochs:   14 | loss: 0.27589
epochs:   15 | loss: 0.27102
epochs:   16 | loss: 0.26418
epochs:   17 | loss: 0.25604
epochs:   18 | loss: 0.24715
epochs:   19 | loss: 0.23719
epochs:   20 | loss: 0.22657


NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7facf9ab5fd0>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [12]:
y_proba

array([[ 0.37983435,  0.62016571],
       [ 0.68942523,  0.31057471],
       [ 0.63028896,  0.36971098],
       ..., 
       [ 0.4139069 ,  0.58609313],
       [ 0.54321432,  0.45678568],
       [ 0.59157926,  0.40842071]], dtype=float32)

In [13]:
np.argmax(y_proba, 1)[:10]

array([1, 0, 0, 0, 1, 0, 0, 1, 0, 0])

In [14]:
y_proba = net.predict_proba(X)

In [15]:
(np.argmax(y_proba, 1) == y.flatten()).mean()

0.92659999999999998

### Grid search

The neural net estimators can be used in conjunction with other sklearn features, such as `GridSearchCV`.

In [16]:
l0 = InputLayer()
l1 = DenseLayer(l0, name='hidden')
l2 = DenseLayer(l1)

In [17]:
net = NeuralNetClassifier(l2, update=SGD())

Parameters are set using the known double-underscore notation, e.g.

`'update__learning_rate': [0.1, 0.5]`.

Note: Instead of having to write

`'layer__incoming__num_units': [50, 100]`

we can just write

`'hidden__num_units': [50, 100]`

because we have given the hidden layer a name, "hidden". This may safe a lot of writing and confusion.

In [18]:
params = {
    'update__learning_rate': [0.1, 0.5],
    'max_epochs': [5, 10],
    'hidden__num_units': [50, 100],
}

In [19]:
cv = GridSearchCV(net, params, scoring='accuracy', refit=False, verbose=3)

In [20]:
cv.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] max_epochs=5, hidden__num_units=50, update__learning_rate=0.1 ...
[CV]  max_epochs=5, hidden__num_units=50, update__learning_rate=0.1, score=0.886623 -   0.3s
[CV] max_epochs=5, hidden__num_units=50, update__learning_rate=0.1 ...
[CV]  max_epochs=5, hidden__num_units=50, update__learning_rate=0.1, score=0.886623 -   0.3s
[CV] max_epochs=5, hidden__num_units=50, update__learning_rate=0.1 ...
[CV]  max_epochs=5, hidden__num_units=50, update__learning_rate=0.1, score=0.881152 -   0.4s
[CV] max_epochs=5, hidden__num_units=50, update__learning_rate=0.5 ...
[CV]  max_epochs=5, hidden__num_units=50, update__learning_rate=0.5, score=0.893221 -   0.3s
[CV] max_epochs=5, hidden__num_units=50, update__learning_rate=0.5 ...
[CV]  max_epochs=5, hidden__num_units=50, update__learning_rate=0.5, score=0.883023 -   0.4s
[CV] max_epochs=5, hidden__num_units=50, update__learning_rate=0.5 ...
[CV]  max_epochs=5, hidden__num_units=50, update_

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   16.9s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7facf9ab5fd0>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=Glor...e=CrossEntropy(eps=1e-12),
          session_kwargs=None, update=SGD(learning_rate=0.01), verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_epochs': [5, 10], 'hidden__num_units': [50, 100], 'update__learning_rate': [0.1, 0.5]},
       pre_dispatch='2*n_jobs', refit=False, scoring='accuracy', verbose=3)

In [21]:
cv.best_params_, cv.best_score_

({'hidden__num_units': 100, 'max_epochs': 10, 'update__learning_rate': 0.1},
 0.89380000000000004)

## Regression task

As is known from sklearn, we have separate estimators for classification and regression.

### Data

In [22]:
X, y = make_regression(n_samples=5000)

Note that apart from using `NeuralNetRegressor` instead of `NeuralNetClassifier`, everything is the same. No need to adjust output nonlinearity or objective.

### Define neural network

In [23]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [24]:
net = NeuralNetRegressor(l2, verbose=1)

In [25]:
net.set_params(update__learning_rate=0.001)

NeuralNetRegressor(batch_iterator=<mink.nolearn.BatchIterator object at 0x7facf9aba400>,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=MeanSquaredError(), session_kwargs=None,
          update=SGD(learning_rate=0.001), verbose=1)

### Train

In [26]:
net.fit(X, y, num_epochs=20)

epochs:    1 | loss: 5004.92871
epochs:    2 | loss: 224.35101
epochs:    3 | loss: 165.94629
epochs:    4 | loss: 129.34683
epochs:    5 | loss: 101.52975
epochs:    6 | loss: 79.79655
epochs:    7 | loss: 62.96289
epochs:    8 | loss: 49.60880
epochs:    9 | loss: 39.12626
epochs:   10 | loss: 30.97226
epochs:   11 | loss: 24.63676
epochs:   12 | loss: 19.71872
epochs:   13 | loss: 15.88611
epochs:   14 | loss: 12.88352
epochs:   15 | loss: 10.56583
epochs:   16 | loss: 8.74349
epochs:   17 | loss: 7.31655
epochs:   18 | loss: 6.17346
epochs:   19 | loss: 5.25926
epochs:   20 | loss: 4.52662


NeuralNetRegressor(batch_iterator=<mink.nolearn.BatchIterator object at 0x7facf9aba400>,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Linear(), num_units=1),
          max_epochs=10, objective=MeanSquaredError(), session_kwargs=None,
          update=SGD(learning_rate=0.001), verbose=1)

## Saving and restoring

### Save previous net

In [27]:
mean_squared_error(y, net.predict(X))

6.0573683828896554

In [28]:
net.save_params('mynet.pkl')

### Create a new net with same architecture

In [29]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [30]:
new_net = NeuralNetRegressor(l2)

In [31]:
new_net.initialize(X, y)

#### score before loading

In [32]:
mean_squared_error(y, new_net.predict(X))

26386.368839208244

In [33]:
new_net.load_params('mynet.pkl')

#### score after loading, the same as saved net

In [34]:
mean_squared_error(y, new_net.predict(X))

6.0573683828896554